In [31]:
import sys
import os

# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add it to sys.path if not already added
if project_root not in sys.path:
    sys.path.append(project_root)

In [32]:
from src.functions.data_generation import DataGeneration
from sklearn.model_selection import train_test_split

data = DataGeneration.generate_data(
    n_samples=10000,
    n_features=20,
    n_informative=4,
    relationship='friedman1',
    noise=0.1,
    random_state=2025
)

# Draw bootstrap sample
bootstrap_data = data.sample(n=1000, replace=True)
bootstrap_data = DataGeneration.add_epsilon(bootstrap_data)

display(bootstrap_data)

X = bootstrap_data.drop(columns=["Target"])
y = bootstrap_data["Target"]

# Split data
X_proper, X_test, y_proper, y_test = train_test_split(X, y, test_size=0.5, random_state=2025)

        
def to_numpy_safe(x):
    return x.to_numpy() if hasattr(x, "to_numpy") else x

# Convert to numpy arrays if needed
X_proper = to_numpy_safe(X_proper)
y_proper = to_numpy_safe(y_proper)
X_test   = to_numpy_safe(X_test)
y_test   = to_numpy_safe(y_test)

,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_11,Feature_12,Feature_13,Feature_14,Feature_15,Feature_16,Feature_17,Feature_18,Feature_19,Target
2436,0.612458,0.302890,0.834216,0.047399,0.719804,0.447398,0.818309,0.146069,0.358147,0.302147,...,0.539084,0.713662,0.197629,0.913003,0.001117,0.667840,0.860411,0.181674,0.673010,0.386551
6656,0.385603,0.467925,0.907015,0.039836,0.588953,0.644273,0.957890,0.212147,0.585204,0.414145,...,0.603729,0.780275,0.357479,0.486714,0.591230,0.767188,0.668701,0.477242,0.001459,0.390648
6474,0.319149,0.809400,0.262848,0.606656,0.283377,0.368345,0.918103,0.772905,0.944256,0.926774,...,0.834927,0.422072,0.096396,0.481261,0.323096,0.594844,0.936251,0.442993,0.626257,0.536076
7529,0.195956,0.451717,0.815696,0.112894,0.054561,0.346277,0.709878,0.955563,0.135598,0.276597,...,0.149316,0.705412,0.797920,0.207436,0.092131,0.423298,0.413823,0.757582,0.139775,0.183488
6601,0.039168,0.180620,0.439045,0.959909,0.322201,0.667473,0.161324,0.580603,0.355139,0.582542,...,0.507440,0.476220,0.624558,0.400699,0.703134,0.464292,0.313551,0.110504,0.620457,0.379691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9114,0.408296,0.599392,0.460550,0.328080,0.424042,0.190567,0.314823,0.250036,0.157833,0.470020,...,0.669130,0.718851,0.850168,0.136981,0.563640,0.060359,0.391233,0.803318,0.353629,0.412264
8556,0.643564,0.655778,0.294223,0.730329,0.794835,0.848344,0.660445,0.097684,0.161359,0.047669,...,0.631116,0.764412,0.483582,0.709741,0.672877,0.672708,0.443215,0.131985,0.542839,0.758849
315,0.162311,0.848049,0.124047,0.895527,0.075379,0.854465,0.842951,0.392911,0.266711,0.724984,...,0.583445,0.319262,0.206723,0.878003,0.256498,0.185100,0.587830,0.654521,0.561543,0.554402
3701,0.516380,0.102074,0.900357,0.725434,0.920071,0.741332,0.269592,0.352505,0.866938,0.102360,...,0.155143,0.003022,0.014080,0.471061,0.782682,0.742665,0.615563,0.353709,0.923567,0.561791


In [29]:
from online_cp.CPS import NearestNeighboursPredictionMachine
from online_cp.CPS import RidgePredictionMachine, KernelRidgePredictionMachine
import numpy as np
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

kernel = C(100) * RBF(length_scale=100)
cps = KernelRidgePredictionMachine(a=1, kernel=kernel)

X_seen = X_proper
y_seen = y_proper

cps.learn_initial_training_set(X_seen, y_seen)

for i, (x, y) in enumerate(zip(X_test, y_test)):
    print(i+1)
    cpd, precomputed = cps.predict_cpd(x=x, return_update=True)
    
    cps.learn_one(x=x, y=y, precomputed=precomputed)
            
    X_seen = np.vstack([X_seen, x])
    y_seen = np.append(y_seen, y)
    
    cps.kernel = C(10) * RBF(length_scale=10)
    cps.a = 0.01
    
    

1
2
3


/Users/simon/Documents/GitHub/Conformal-Predictive-Decision-Making/.venv/lib/python3.13/site-packages/online_cp/CPS.py:419: RuntimeWarning: invalid value encountered in sqrt
  sqrt_one_minus_h = np.sqrt(1 - h[:-1])
/Users/simon/Documents/GitHub/Conformal-Predictive-Decision-Making/.venv/lib/python3.13/site-packages/online_cp/CPS.py:420: RuntimeWarning: invalid value encountered in sqrt
  A = np.dot(H[-1, :-1], y) / np.sqrt(1 - h[-1])  + (y - H[:-1, :-1] @ y) / sqrt_one_minus_h
/Users/simon/Documents/GitHub/Conformal-Predictive-Decision-Making/.venv/lib/python3.13/site-packages/online_cp/CPS.py:421: RuntimeWarning: invalid value encountered in sqrt
  B = np.sqrt(1 - h[-1]) * np.ones(n-1) + H[-1, :-1] / sqrt_one_minus_h


AssertionError: C contains NaN values

In [34]:
X_seen = X_proper
y_seen = y_proper

kernel = C(100) * RBF(length_scale=100)
cps = KernelRidgePredictionMachine(a=0.01, kernel=kernel)

cps.learn_initial_training_set(X_seen, y_seen)

for i, (x, y) in enumerate(zip(X_test, y_test)):
    print(i+1)
    cpd, precomputed = cps.predict_cpd(x=x, return_update=True)
    
    kernel = C(100) * RBF(length_scale=100)
    cps = KernelRidgePredictionMachine(a=0.01, kernel=kernel)
            
    X_seen = np.vstack([X_seen, x])
    y_seen = np.append(y_seen, y)
    
    cps.learn_initial_training_set(X_seen, y_seen)
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [16]:
X_seen = np.array([[0.61690403, 0.87447608, 0.46287463, 0.38878679, 0.66070327,
        0.00232492, 0.54601537, 0.64432384, 0.68427912, 0.62110265,
        0.59712489, 0.6877667 , 0.35055853, 0.58650853, 0.80618439,
        0.29815741, 0.735198  , 0.60935794, 0.92569768, 0.11617104],
       [0.91579433, 0.02432272, 0.87300042, 0.21273217, 0.23742334,
        0.4261802 , 0.62978608, 0.44619609, 0.08229804, 0.99548367,
        0.05369916, 0.49441106, 0.08500298, 0.55486127, 0.61009087,
        0.85256786, 0.10284571, 0.19523638, 0.33010014, 0.4991184 ],
       [0.26023365, 0.07644817, 0.27885393, 0.58315717, 0.49324211,
        0.60589993, 0.32866339, 0.83777911, 0.00929254, 0.81334764,
        0.74754069, 0.4462899 , 0.46590148, 0.52706113, 0.71749153,
        0.63081901, 0.57733303, 0.59577937, 0.11531968, 0.73717042],
       [0.45076311, 0.78577283, 0.33347904, 0.85423709, 0.02998891,
        0.9413173 , 0.00685749, 0.3835995 , 0.26137372, 0.97404553,
        0.018884  , 0.38371018, 0.29799676, 0.04687279, 0.74581966,
        0.67774238, 0.43461648, 0.60646479, 0.82850803, 0.58361229],
       [0.1975792 , 0.8138122 , 0.81859487, 0.8338643 , 0.70860933,
        0.445699  , 0.50907458, 0.28168284, 0.27746932, 0.20906813,
        0.58993412, 0.78687081, 0.50354748, 0.62500309, 0.50526637,
        0.49038134, 0.23918682, 0.22314794, 0.94578918, 0.34079787],
       [0.38876141, 0.50926379, 0.30521151, 0.58781281, 0.45184239,
        0.74458284, 0.35833841, 0.27592876, 0.05097637, 0.90305267,
        0.71552171, 0.1175358 , 0.80581814, 0.89393874, 0.88091416,
        0.63382612, 0.06135225, 0.53220251, 0.15069796, 0.23486857],
       [0.03530995, 0.67016732, 0.59231133, 0.95861001, 0.61408845,
        0.41297966, 0.54875759, 0.93992849, 0.11189206, 0.55769144,
        0.32024245, 0.25934737, 0.68644511, 0.57964617, 0.40294091,
        0.88371262, 0.74555981, 0.61305455, 0.77577919, 0.33944854],
       [0.46670223, 0.91631504, 0.47087016, 0.18015492, 0.41974278,
        0.28584777, 0.84607481, 0.86991923, 0.42356697, 0.15495714,
        0.9474776 , 0.8749199 , 0.22955438, 0.79390101, 0.14493672,
        0.37139354, 0.65026267, 0.10772185, 0.80760986, 0.49279778],
       [0.32678263, 0.9544731 , 0.55092026, 0.38795748, 0.8352388 ,
        0.00210927, 0.33727855, 0.26595158, 0.40630893, 0.41415723,
        0.0052716 , 0.70843283, 0.41831813, 0.66410819, 0.53172061,
        0.83529665, 0.03425966, 0.32402458, 0.78408303, 0.12906995],
       [0.49775345, 0.98513313, 0.44786847, 0.97486652, 0.87163405,
        0.94023883, 0.18774018, 0.51279296, 0.12047815, 0.94631978,
        0.85693063, 0.81514435, 0.80231037, 0.48906128, 0.56876531,
        0.99189387, 0.45257105, 0.10296499, 0.74130788, 0.31881868],
       [0.57737144, 0.34163632, 0.11251146, 0.93713594, 0.85086369,
        0.5350091 , 0.07863634, 0.84201752, 0.59004941, 0.55503539,
        0.21264559, 0.31007106, 0.92480549, 0.63227108, 0.37677974,
        0.8794214 , 0.17776382, 0.3611084 , 0.74999908, 0.50469715],
       [0.44499032, 0.20012612, 0.90493299, 0.32613187, 0.17479493,
        0.84286145, 0.4485736 , 0.06452242, 0.51267103, 0.62314745,
        0.1255175 , 0.50428765, 0.69856286, 0.59555401, 0.96334816,
        0.52138717, 0.91176839, 0.90389543, 0.14513848, 0.68142977],
       [0.34648455, 0.27203819, 0.38153724, 0.69674206, 0.49339642,
        0.66944446, 0.07806471, 0.10273499, 0.75633336, 0.38110762,
        0.13513837, 0.29433877, 0.48950473, 0.54120231, 0.50687474,
        0.50381298, 0.47774431, 0.84202576, 0.05316951, 0.27947344],
       [0.41746397, 0.36938514, 0.54905781, 0.51777302, 0.42910892,
        0.46886277, 0.15823245, 0.36733995, 0.82205883, 0.30137097,
        0.84500858, 0.42169599, 0.07131294, 0.17719126, 0.26603235,
        0.35291606, 0.2957825 , 0.78882948, 0.88551834, 0.72960781],
       [0.64478003, 0.13189925, 0.53105197, 0.53013952, 0.58808762,
        0.94376259, 0.4303951 , 0.61367101, 0.04403258, 0.75806092,
        0.43616002, 0.93574578, 0.60334329, 0.61923449, 0.38394202,
        0.6685584 , 0.85260882, 0.59097694, 0.20951957, 0.0786221 ],
       [0.7552315 , 0.4614683 , 0.26357669, 0.09284826, 0.60315382,
        0.31369358, 0.32974014, 0.19993976, 0.26682244, 0.20350981,
        0.97080258, 0.68341552, 0.37141491, 0.00279919, 0.46767294,
        0.08456203, 0.66196791, 0.0471992 , 0.59846991, 0.81362607],
       [0.05628668, 0.49687508, 0.29712254, 0.95553429, 0.61047986,
        0.08262895, 0.13865336, 0.23443317, 0.12284737, 0.6011743 ,
        0.78511282, 0.17012297, 0.31277596, 0.70212678, 0.20009298,
        0.50781627, 0.46642926, 0.85430861, 0.86508383, 0.97927647],
       [0.11297176, 0.3193713 , 0.69476313, 0.52214091, 0.86160845,
        0.04017112, 0.41635022, 0.21291641, 0.47074962, 0.25049169,
        0.22260359, 0.32668977, 0.16654484, 0.35965967, 0.69289228,
        0.77995073, 0.67096882, 0.68037106, 0.50950037, 0.50659541],
       [0.80012599, 0.94870053, 0.59182567, 0.58563817, 0.57371547,
        0.31348861, 0.72225411, 0.24891447, 0.31003646, 0.7514819 ,
        0.55619309, 0.95382337, 0.83846726, 0.98657012, 0.05288109,
        0.6189602 , 0.09206017, 0.25383892, 0.07908283, 0.42123578],
       [0.18461492, 0.6433796 , 0.85085628, 0.079005  , 0.03579918,
        0.83816129, 0.79430468, 0.98249415, 0.96392254, 0.47197439,
        0.40337412, 0.98575147, 0.58227895, 0.47064495, 0.1345221 ,
        0.33841849, 0.09543069, 0.72290251, 0.99462444, 0.88337705],
       [0.10535415, 0.31401669, 0.89019704, 0.95075513, 0.98326673,
        0.85107112, 0.90997526, 0.34222438, 0.5872768 , 0.14046719,
        0.47812487, 0.86021296, 0.94306024, 0.93131597, 0.04829842,
        0.19960255, 0.91757876, 0.48683352, 0.3372579 , 0.12750855],
       [0.189923  , 0.76886905, 0.40357378, 0.72013392, 0.38589263,
        0.52346177, 0.05894595, 0.5501359 , 0.21885853, 0.19682074,
        0.74164482, 0.480565  , 0.32364022, 0.72340493, 0.50971304,
        0.66224582, 0.96321424, 0.53279458, 0.0367493 , 0.47548291],
       [0.98184061, 0.82242742, 0.91311657, 0.19905862, 0.8235858 ,
        0.86206807, 0.366947  , 0.3621475 , 0.1210519 , 0.96746118,
        0.23936093, 0.31172959, 0.04285227, 0.26997335, 0.07257182,
        0.63628677, 0.76711326, 0.67381972, 0.78972515, 0.76433807],
       [0.87830652, 0.40247937, 0.84492561, 0.54248177, 0.35728949,
        0.65521079, 0.23707085, 0.94646998, 0.80260709, 0.0690497 ,
        0.46439782, 0.60276499, 0.85467219, 0.62476244, 0.68448804,
        0.91727919, 0.56170773, 0.78719359, 0.17924195, 0.04227915],
       [0.9164081 , 0.24035337, 0.30209463, 0.21477177, 0.08174764,
        0.47510183, 0.4718107 , 0.37238835, 0.08139098, 0.6180113 ,
        0.59167568, 0.09392855, 0.95334931, 0.52972731, 0.25937394,
        0.60819322, 0.5763105 , 0.65603732, 0.12963586, 0.5527947 ],
       [0.73615201, 0.61638444, 0.39035279, 0.55641677, 0.62421441,
        0.25802111, 0.93876485, 0.12806208, 0.34472932, 0.21058852,
        0.48089751, 0.01945183, 0.68033345, 0.50813111, 0.67703167,
        0.20175018, 0.08511787, 0.12465513, 0.61166981, 0.72810863],
       [0.38814616, 0.33016759, 0.8347916 , 0.31759653, 0.5394995 ,
        0.38193218, 0.6728284 , 0.90742288, 0.96309925, 0.32354187,
        0.41797015, 0.85598548, 0.59349483, 0.38311874, 0.05004716,
        0.69893119, 0.47553582, 0.45554697, 0.52498651, 0.92197788],
       [0.83335522, 0.18028557, 0.83925735, 0.88395057, 0.52634498,
        0.75802361, 0.35227691, 0.75901604, 0.36283574, 0.66703115,
        0.55953195, 0.41380416, 0.40722059, 0.97550619, 0.82347904,
        0.80074657, 0.29144675, 0.16375034, 0.90418439, 0.84182748],
       [0.63245495, 0.80247196, 0.80860457, 0.54979344, 0.26280457,
        0.99507387, 0.4016898 , 0.79887646, 0.67784378, 0.72811244,
        0.48766279, 0.88348734, 0.00211905, 0.35123269, 0.50311107,
        0.22038668, 0.79147001, 0.78738263, 0.0484944 , 0.41137662],
       [0.62818502, 0.01500276, 0.11845419, 0.73915755, 0.72855623,
        0.66182345, 0.64051285, 0.5690818 , 0.31887823, 0.71412555,
        0.85200606, 0.46907468, 0.96604888, 0.16398006, 0.31084425,
        0.54640548, 0.57224212, 0.63623418, 0.32846183, 0.35210896],
       [0.66463358, 0.489743  , 0.45137743, 0.65596187, 0.87487342,
        0.96535247, 0.08441544, 0.84104069, 0.91777603, 0.6231457 ,
        0.43552652, 0.03328698, 0.87287412, 0.09696855, 0.7248814 ,
        0.66677396, 0.16109435, 0.70440369, 0.62900332, 0.7124535 ],
       [0.40681158, 0.58735837, 0.6275798 , 0.68556781, 0.16984881,
        0.60399803, 0.71763298, 0.50561357, 0.87472338, 0.12593806,
        0.10358275, 0.75243194, 0.76829163, 0.08220347, 0.19204259,
        0.20552235, 0.59046966, 0.5074028 , 0.11196002, 0.52498511],
       [0.46256204, 0.80895007, 0.07956222, 0.34577538, 0.4359663 ,
        0.59042979, 0.7997986 , 0.41417156, 0.98475942, 0.20623434,
        0.95571796, 0.54303571, 0.98554355, 0.70865926, 0.80314321,
        0.75906356, 0.6162506 , 0.69717804, 0.06130025, 0.82293526],
       [0.65224292, 0.69759562, 0.38363283, 0.41599009, 0.50379321,
        0.08028608, 0.13806171, 0.72361662, 0.01821525, 0.02295151,
        0.90915442, 0.82481228, 0.34934783, 0.56330673, 0.82881332,
        0.61034537, 0.02458505, 0.33239802, 0.46095784, 0.12674915],
       [0.81891762, 0.17742008, 0.5583408 , 0.67670142, 0.06625006,
        0.77871585, 0.99401999, 0.18710728, 0.12540084, 0.28348024,
        0.84032088, 0.07620544, 0.05674359, 0.85768277, 0.53645601,
        0.01023173, 0.7079834 , 0.9916835 , 0.73517333, 0.26736103],
       [0.99995419, 0.98983132, 0.92818045, 0.09891396, 0.71226002,
        0.8487543 , 0.78082758, 0.13319967, 0.93595417, 0.07502853,
        0.18302575, 0.71907877, 0.2610095 , 0.51160147, 0.55268635,
        0.42863181, 0.88749287, 0.29829858, 0.45829795, 0.60283157],
       [0.91784771, 0.62774378, 0.08959139, 0.12344046, 0.94897967,
        0.98446017, 0.10304698, 0.69274853, 0.65948703, 0.96761688,
        0.84741886, 0.71584026, 0.08163966, 0.12905297, 0.40769082,
        0.12751238, 0.31982213, 0.12098797, 0.42515927, 0.14501096],
       [0.50014376, 0.25764066, 0.89321616, 0.42712926, 0.0554846 ,
        0.95114228, 0.71358808, 0.28102892, 0.16923638, 0.07674369,
        0.81212113, 0.36325585, 0.86309813, 0.71734953, 0.94592495,
        0.97832059, 0.99662547, 0.27319607, 0.45119052, 0.96076004],
       [0.90253823, 0.24733245, 0.32683554, 0.52682568, 0.49748272,
        0.12236442, 0.03298476, 0.20974979, 0.16296402, 0.58427078,
        0.26675697, 0.46668716, 0.84086689, 0.64536565, 0.3106762 ,
        0.11220032, 0.8994947 , 0.73787026, 0.6554433 , 0.6753808 ],
       [0.18567906, 0.00399292, 0.47641247, 0.91865786, 0.41887556,
        0.70834108, 0.17959301, 0.4380631 , 0.67724269, 0.69667343,
        0.72800649, 0.17028013, 0.9146416 , 0.92159125, 0.41425331,
        0.4006198 , 0.36910244, 0.63384678, 0.84616134, 0.17025744],
       [0.92531473, 0.1655071 , 0.51796206, 0.21671123, 0.54801563,
        0.27716926, 0.14992817, 0.05253494, 0.66211856, 0.68645151,
        0.78970834, 0.35615426, 0.49116681, 0.94477949, 0.93507594,
        0.47452467, 0.70245381, 0.43934428, 0.95295703, 0.4187147 ],
       [0.33515453, 0.10705191, 0.25710262, 0.36302646, 0.82550822,
        0.51990847, 0.72108586, 0.74177152, 0.11450249, 0.89406315,
        0.34797566, 0.895625  , 0.46165193, 0.2135426 , 0.53488725,
        0.72321418, 0.7878882 , 0.30746976, 0.76429345, 0.73536041]])
y_seen = np.array([0.57218823, 0.20232463, 0.31767826, 0.62873191, 0.64141939,
       0.49516018, 0.4578338 , 0.46423245, 0.55617846, 0.84011891,
       0.78049052, 0.33286834, 0.4194628 , 0.39678775, 0.35077619,
       0.47188588, 0.48166685, 0.37978283, 0.53653371, 0.22448738,
       0.63234304, 0.45718868, 0.51364395, 0.63637147, 0.3139276 ,
       0.64666755, 0.40074023, 0.63491258, 0.63916471, 0.48231435,
       0.66133472, 0.50648569, 0.62976103, 0.57465927, 0.37493874,
       0.27508023, 0.64724572, 0.37859445, 0.49631768, 0.37113892,
       0.31092492, 0.32695493])
x = np.array([0.09842664, 0.91452938, 0.42056915, 0.46360808, 0.65245194,
       0.83982535, 0.78429744, 0.49834649, 0.28574984, 0.87174272,
       0.61163489, 0.17329679, 0.46724292, 0.87860093, 0.52903206,
       0.24919905, 0.98664782, 0.51081124, 0.82706065, 0.17630974])
y = np.float64(0.3558477174746414)

In [17]:
kernel = C(10) * RBF(length_scale=100)
cps = KernelRidgePredictionMachine(a=0.01, kernel=kernel)

cps.learn_initial_training_set(X_seen, y_seen)

cpd, precomputed = cps.predict_cpd(x=x, return_update=True)